In [47]:
import joblib
import torch
import tqdm

from data import Channel as Ch
import torch_simple_unet as unet
from torchmetrics import Dice

In [13]:
y_hat_ens = joblib.load("pred.joblib")
y_hat_ens = [torch.cat(yi) for yi in y_hat_ens]
y_hat_ens = torch.stack(y_hat_ens)
y_hat_ens.shape

torch.Size([10, 21150, 64, 64])

In [12]:
_, _, ds_test = unet.get_dataset(use_channels=None, random_seed=1337)
ds_test.load()

Loading data into memory...


In [50]:
lsm = [xi[Ch.IS_LAND] for (xi, _) in tqdm.tqdm(ds_test)]
lsm = torch.stack(lsm)
lsm.shape

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21150/21150 [00:12<00:00, 1651.19it/s]


torch.Size([21150, 64, 64])

In [56]:
# Postproc predictions: Force land values to zero
is_sea = 1 - lsm
y_hat_ens = y_hat_ens * is_sea

In [21]:
y = [yi for (_, yi) in tqdm.tqdm(ds_test)]
y = torch.stack(y)
y.shape

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21150/21150 [00:11<00:00, 1872.06it/s]


torch.Size([21150, 64, 64])

In [57]:
n, _, _, _ = y_hat_ens.shape
for i in range(n):
    d = Dice()(y_hat_ens[i], y)
    print(d)

tensor(0.6412)
tensor(0.6405)
tensor(0.6206)
tensor(0.6528)
tensor(0.6475)
tensor(0.6057)
tensor(0.6434)
tensor(0.6372)
tensor(0.6376)
tensor(0.6160)


In [58]:
Dice()(y_hat_ens.mean(dim=0), y)

tensor(0.6480)

In [59]:
y_hat_ens.std(dim=0).shape

torch.Size([21150, 64, 64])